# Objetivo do Notebook

Neste arquivo, o objetivo é configurar uma API para uso do modelo e executá-la

# 0.Imports

In [43]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
import requests

# 1. Configurando a API

- #### Endpoint

> POST /predict

- #### Descrição
Esse endpoint recebe uma requisição com um vetor de características (features) sobre duas pessoas (V1 e V2) e retorna a predição da probabilidade de ligação entre elas, segundo o modelo treinado.

- #### Formato do JSON de entrada
O JSON enviado deve conter um único campo chamado "data", cujo valor é uma lista ordenada de 20 elementos, representando as seguintes variáveis:

| Ordem | Nome                            | Tipo    | Descrição                                            |
| ----- | ------------------------------- | ------- | ---------------------------------------------------- |
| 1     | V1                              | `int`   | ID numérico da primeira pessoa                       |
| 2     | V2                              | `int`   | ID numérico da segunda pessoa                        |
| 3     | grau                            | `str`   | Grau de relacionamento (ex: `"pai"`, `"irmão"`)      |
| 4     | proximidade                     | `str`   | Nível de proximidade (ex: `"próximo"`, `"distante"`) |
| 5     | idade\_V1                       | `int`   | Idade da pessoa V1                                   |
| 6     | estado\_civil\_V1               | `str`   | Estado civil de V1 (ex: `"solteiro"`, `"casado"`)    |
| 7     | qt\_filhos\_V1                  | `int`   | Quantidade de filhos de V1                           |
| 8     | estuda\_V1                      | `int`   | Estuda? 1 = Sim, 0 = Não                             |
| 9     | trabalha\_V1                    | `int`   | Trabalha? 1 = Sim, 0 = Não                           |
| 10    | pratica\_esportes\_V1           | `int`   | Pratica esportes? 1 = Sim, 0 = Não                   |
| 11    | transporte\_mais\_utilizado\_V1 | `str`   | Meio de transporte mais usado por V1 (ex: `"carro"`) |
| 12    | IMC\_V1                         | `float` | Índice de Massa Corporal (IMC) de V1                 |
| 13    | idade\_V2                       | `int`   | Idade da pessoa V2                                   |
| 14    | estado\_civil\_V2               | `str`   | Estado civil de V2                                   |
| 15    | qt\_filhos\_V2                  | `int`   | Quantidade de filhos de V2                           |
| 16    | estuda\_V2                      | `int`   | Estuda? 1 = Sim, 0 = Não                             |
| 17    | trabalha\_V2                    | `int`   | Trabalha? 1 = Sim, 0 = Não                           |
| 18    | pratica\_esportes\_V2           | `int`   | Pratica esportes? 1 = Sim, 0 = Não                   |
| 19    | transporte\_mais\_utilizado\_V2 | `str`   | Meio de transporte mais usado por V2                 |
| 20    | IMC\_V2                         | `float` | Índice de Massa Corporal (IMC) de V2                 |

- #### Exemplo de JSON válido:

{
  "data": [1, 2, "pai", "próximo", 32, "solteiro", 0, 1, 1, 1, "carro", 24.5,
           29, "casado", 1, 0, 1, 0, "ônibus", 22.3]
}

- #### Resposta (JSON)

{
  "prediction": 0.0573
}
prediction: Valor predito pelo modelo (probabilidade estimada de relacionamento entre V1 e V2).

- #### Possíveis erros

| Código | Motivo                                        |
| ------ | --------------------------------------------- |
| 400    | JSON malformado ou quantidade errada de dados |
| 500    | Erro interno do modelo ou do processamento    |

# 2. Executando a API

Executando a API no terminal

In [ ]:
url = 'http://127.0.0.1:8000/predict'

# 3. Testes

In [ ]:
# Teste com JSON em formato correto

payload = {
    "data": [10, 25, "irmão", "distante", 40, "casado", 2, 0, 1, 0, "bicicleta", 27.8,
             35, "solteiro", 0, 1, 1, 1, "metrô", 23.1]
}
response = requests.post(url, json=payload)
print("Teste 1:", response.json())

Teste 1: {'prediction': 0.3772524297237396}


In [ ]:
# Teste com JSON em formato correto

payload = {
    "data": [100, 200, "tio", "muito próximo", 99, "divorciado", 8, 0, 0, 1, "ônibus", 18.0,
             5, "solteiro", 0, 1, 0, 1, "carro", 30.0]
}
response = requests.post(url, json=payload)
print("Teste 2:", response.json())

Teste 2: {'prediction': 0.7564266324043274}


In [ ]:
# Teste de JSON inválido (campo 'data' ausente)

payload = {
    "input": [1, 2, "pai", "próximo", 32, "solteiro", 0, 1, 1, 1, "carro", 24.5,
              29, "casado", 1, 0, 1, 0, "ônibus", 22.3]
}
response = requests.post(url, json=payload)
print("Teste 3:", response.json())

Teste 3: {'error': "JSON deve conter o campo 'data'"}


In [ ]:
# Teste de JSON inválido (quantidade incorreta de features, 19 ao invés de 20)

payload = {
    "data": [1, 2, "pai", "próximo", 32, "solteiro", 0, 1, 1, 1, "carro", 24.5,
             29, "casado", 1, 0, 1, 0, "ônibus"]  # Faltando IMC_V2
}
response = requests.post(url, json=payload)
print("Teste 4:", response.json())

Teste 4: {'error': 'Esperado 20 features, recebido 19'}


In [ ]:
# Teste de JSON inválido (idade como string)

payload = {
    "data": [1, 2, "pai", "próximo", "trinta e dois", "solteiro", 0, 1, 1, 1, "carro", 24.5,
             29, "casado", 1, 0, 1, 0, "ônibus", 22.3]
}
response = requests.post(url, json=payload)
print("Teste 5:", response.json())

Teste 5: {'error': "Cannot use mean strategy with non-numeric data:\ncould not convert string to float: 'trinta e dois'"}


In [51]:
# Teste com JSON em formato correto

payload = {
    "data": [1, 2, "amigo", "distante", None, "casado", 2, 1, 1, 0, "carro", None,
             35, "solteiro", 0, 0, 1, 1, "bicicleta", 25.0]
}
response = requests.post(url, json=payload)
print("Teste 6:", response.json())

Teste 6: {'prediction': 0.3956749439239502}
